In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def reset_graph(seed=318):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

In [ ]:
pic_width = 28
pic_height = 28
pic_size = pic_width*pic_height

filter_height = 5
filter_width = 5
batch_size = 10
n_epochs = 100
pic_class = 10

n_kernels_conv1 = 32
n_kernels_conv2 = 64

n_inputs_full1 = 7*7*n_kernels_conv2

n_neurons_dense1 = 24
n_neurons_dense2 = 10

n_channels = 1

In [ ]:
reset_graph()

In [ ]:
x = tf.placeholder(tf.float32,[None,pic_size])
y = tf.placeholder(tf.float32,[None,pic_class])
keep_prob = tf.placeholder(tf.float32)

In [ ]:
def conv_layer(x,n_kennels,activation=None):
    n_inputs = int(x.get_shape()[-1])
    stddev = 2/np.sqrt(n_inputs)
    W_init = tf.truncated_normal([filter_height,filter_width,n_inputs,n_kennels],stddev=stddev)
    b_init = tf.zeros([n_kennels],tf.float32)
    W = tf.Variable(W_init)
    b = tf.Variable(b_init)
    
#     convolution
    conv = tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
    sigma = conv + b
    
    if activation is not None:
        act = activation(sigma)
    else:
        act = sigma
    
    pool = tf.nn.max_pool(act,ksize=[1,2,2,1],strides=[1,1,1,1],padding='SAME')
    
    return pool

In [ ]:
def dense_layer(x,n_neurons,activation=None):
    n_inputs = int(x.get_shape()[-1])
    stddev = 2/np.sqrt(n_inputs)
    W_init = tf.truncated_normal([n_inputs,n_neurons],stddev=stddev)
    b_init = tf.zeros([n_neurons])
    W = tf.Variable(W_init)
    b = tf.Variable(b_init)
    
    dense = tf.matmul(x,W) + b
    
    if activation is not None:
        act = activation(dense)
    else:
        act = dense
    
    return act

In [ ]:
x_conv = tf.reshape(x,[-1,pic_height,pic_width,n_channels])
conv1 = conv_layer(x_conv,n_kernels_conv1,tf.nn.relu)
conv2 = conv_layer(conv1,n_kernels_conv2,tf.nn.relu)

x_dense = tf.reshape(conv2,[-1,n_inputs_full1])

dense1 = dense_layer(x_dense,n_neurons_dense1,tf.nn.tanh)
prediction = dense_layer(dense1,n_neurons_dense2,tf.nn.softmax)

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [ ]:
correct = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [ ]:
loss_summary = tf.summary.scalar('loss',loss)
acc_summary = tf.summary.scalar('acc',accuracy)

merged = tf.summary.merge_all()

log_dir = 'logdir'
file_writer = tf.summary.FileWriter(log_dir,tf.get_default_graph())

In [ ]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for epoch in range(n_epochs):
        train_acc = 0.0
        test_acc = 0.0
        x_batch,y_batch = mnist.train.next_batch(batch_size)
        merged_str,acc,_ = sess.run([merged,accuracy,train],feed_dict={x:x_batch,y:y_batch,keep_prob:0.7})
        train_acc += acc
        x_batch,y_batch = mnist.test.next_batch(batch_size)
        acc = sess.run(accuracy,feed_dict={x:x_batch,y:y_batch,keep_prob:1.0})
        test_acc += acc
        file_writer.add_summary(merged_str,epoch)
        print("Epoch:"+ str(epoch) + "train_acc:" + str(train_acc) +  "train_acc:" + str(test_acc))
    file_writer.close()